## 이 문서를 수정할 당신에게...
#### 현재 상황은 아래와 같습니다.
1. Assistant와 Agent의 성능 측정을 위해서는 이에 대한 evaluation 제작이 필요합니다.

#### 당신의 목표는 아래와 같습니다.
1. [generate dialog 노트북](generate_dialog.ipynb)에서 생성된 대화 데이터셋을 받아서 Assistant에 대한 evaluation set을 구성합니다.
2. Agent의 evaluation을 수행할 수 있는 방법을 생각하고, evaluation set을 제작합니다.(Tool에서 데이터를 잘 빼오는지에 대한 evaluation이 따로 필요할 수 있습니다.)


#### 생각할 수 있는 해결 방식은 아래와 같습니다.
문제 1에 대하여...
1. Format 설정
어떤 format으로 evaluation을 작성할지 먼저 생각합니다. 아래는 format의 예시입니다.
```
conversation:
   이우선: 안녕하세요! 저는 이우선이라고 합니다. 와인에 대해 궁금한 점이 있으시면 언제든지 물어보세요! 😊<END_OF_TURN>
    User: 와인 추천해주세요 <END_OF_TURN>
    이우선: 좋아요! 와인 추천을 위해 몇 가지 질문을 드리겠습니다. 먼저, 어떤 종류의 와인을 선호하시나요?<END_OF_TURN>
    User: 레드 와인이요 <END_OF_TURN>
stage_history: "1, 2"
answer: 2
```
2. LLM 활용하기
LLM이 이러한 annotation을 하는데 특화되어 있습니다. LLM을 활용하여 일단 제작한 이후 사람이 검수하는 방식을 사용하면 빠른 수행이 될 것 으로 생각됩니다.

assistant 단계가 확정지어지고, 대화 데이터셋 생성이 마무리 된 후에 이를 수행할 수 있을 듯 합니다.


#### 참고 사항
아래 논문은 직접적이진 않지만 비싼 사람 데이터대신 언어모델을 사용하여 데이터를 수집하는 방식에 근간이 된 논문입니다. 이 논문이 도움이 될 수도 있습니다. 
- [Self-Instruct: Aligning Language Models with Self-Generated Instructions](https://arxiv.org/pdf/2212.10560.pdf)

추가로 이러한 작업을 하는 방식에 대한 논문도 많을 것으로 생각되니 논문을 먼저 찾아보는 것도 좋습니다.

-------------------------------------------------------------

### Assistant evaluation 제작

[대화 데이터셋 파일](./templates/examples/dialog_generate_prompt_examples.yaml)로 부터 [assistant evaluation 파일](./evaluation_data/assistant_evaluation.yaml)을 제작합니다.

단계는 아래와 같습니다.
1. 수집된 대화 데이터셋을 기반으로 대화를 자릅니다.
2. 사람이 직접 annotation을 지정합니다.

Assistant의 단계는 아래와 같습니다.
1. 소개: 대화를 시작하거나 고객의 첫 응답에 대답할 때 이 단계를 거칩니다. 자신을 소개하여 대화를 시작하세요. 공손하고 예의 바른 태도를 유지하면서 전문적인 대화 톤을 유지하세요.
2. 니즈 분석: 고객이 상품이나 장소 추천을 원할 때, 고객이 무엇을 원하는지 파악되지 않았으면 고객 니즈를 파악하기 위해 질문하는 단계입니다. 상품 추천의 경우 고객이 원하는 맛, 향 그리고 가격대 중 어떠한 것이라도 파악되지 않았다면 이 단계를 계속 진행합니다. 장소 추천의 경우 가고자하는 위치를 질문을 통해 파악합니다.
3. 상품 추천: 니즈 분석이 끝난 후, 고객이 원하는 적합한 와인을 추천하는 단계입니다. 고객이 원하는 가격대는 반드시 파악되어야합니다.
4. 판매: 고객이 추천한 상품을 원할 때 그 특정 상품에 대해 구체적으로 설명하는 단계입니다.
5. 장소 추천: 고객이 가고자 하는 위치를 참고하여 주변 와인 바를 추천하는 단계입니다.
6. 마무리: 작업을 마치면 고객에게 작별 인사를 전합니다.
7. 질문 및 답변: 고객의 일반적인 지식이나 상식, 혹은 정보에 관한 질문에 정확하고 자세한 답변을 제공해야하는 단계입니다.
8. 주어진 단계에 해당하지 않음: 1에서 7까지의 단계 중 어느 하나도 해당되지 않을 때 사용되는 단계입니다.

In [19]:
from ruamel.yaml import YAML
import re

In [26]:
yaml = YAML()
yaml.indent(mapping=4, sequence=4)
yaml.default_flow_style = False
yaml.preserve_quotes = True

# ruamel.yaml을 사용하여 입력 YAML 파일을 파싱합니다.
with open('./templates/examples/dialog_generate_prompt_examples.yaml', 'r', encoding='utf-8') as f:
    data = yaml.load(f)

output_data = []

for item in data:
    # 대화를 줄바꿈('\n') 단위로 분할합니다.
    lines = item['conversation'].split('\n')

    for i in range(1, len(lines), 2):
        # 현재의 대화 단계를 가져와서 output_data에 추가합니다.
        output_item = {}
        
        conversation = '\n'.join(lines[:i]).strip() + '\n'
        conversation = conversation.replace('유저:', 'User:')
        conversation = conversation.replace('어시스턴트:', '이우선:')
        conversation = conversation.replace('\n', '<END_OF_TURN>\n')
        output_item['conversation'] = conversation
        output_item['answer'] = '\n'
        output_data.append(output_item)

# ruamel.yaml을 사용하여 output_data를 새로운 YAML 파일에 쓰기
with open('./evaluation_data/assistant_evaluation.yaml', 'w', encoding='utf-8') as f:
    yaml.dump(output_data, f)
